In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Input, InputLayer, Dropout, Dense, Flatten, Embedding, Add, Concatenate
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop

# import torch
from sklearn.metrics import precision_recall_fscore_support

In [2]:
# Good data
# df = pd.read_csv('./data/kokil dec 6 reprepare/affcon_final_with_linguistics.csv')

In [3]:
# Messy data
# df = pd.read_csv('./data/affcon_final_politeness_strategies_merge.csv')

In [4]:
# Data with Throughput & WorkTime
df = pd.read_csv('./data/kokil dec 6 reprepare/conf_pc_worker_sem.csv')

In [5]:
df = df.dropna()

In [6]:
# # Throughput (weighted by variance)
# df_throughput = df[['Throughput.1', 'Throughput.2', 'Throughput.3', 'Throughput.4', 'Throughput.5']].copy()
# df_throughput['var_throughput'] = df_throughput.var(axis=1)
# df_throughput['var_throughput'] = df_throughput['var_throughput'] / df_throughput['var_throughput'].max()

#Throughput (weighted by inverted U-shape variance)
df_throughput = df[['Throughput.1', 'Throughput.2', 'Throughput.3', 'Throughput.4', 'Throughput.5']].copy()
df_throughput['var_throughput'] = df_throughput.var(axis=1)
max_val = df_throughput['var_throughput'].max()
min_val = df_throughput['var_throughput'].min()
tp_mid = ((max_val - min_val) // 2) + min_val
tp_to_list = df_throughput['var_throughput'].tolist()

u_shaped_variance = []
for each in tp_to_list:
    if each > tp_mid:
        u_shaped_variance.append(2*tp_mid - each)
    else:
        u_shaped_variance.append(each)
        
df_throughput['var_throughput_u_shaped'] = u_shaped_variance
df_throughput['var_throughput_u_shaped'] = df_throughput['var_throughput_u_shaped'] / df_throughput['var_throughput_u_shaped'].max()
df_throughput = df_throughput.assign(var_throughput=df_throughput['var_throughput_u_shaped'])

# # WorkTime (weighted by variance)
df_worktime = df[['WorkTime.1', 'WorkTime.2', 'WorkTime.3', 'WorkTime.4', 'WorkTime.5']].copy()
df_worktime['var_worktime'] = df_worktime.var(axis=1)
df_worktime['var_worktime'] = df_worktime['var_worktime'] / df_worktime['var_worktime'].max()

In [7]:
df_throughput

,Throughput.1,Throughput.2,Throughput.3,Throughput.4,Throughput.5,var_throughput,var_throughput_u_shaped
5,390,2331,193,2139,162.0,0.743479,0.743479
6,234,2134,2331,1022,1411.0,0.451285,0.451285
7,521,2139,2660,1411,892.0,0.479614,0.479614
8,2632,2415,1482,892,3747.0,0.753045,0.753045
9,2905,95,528,892,300.0,0.801213,0.801213
...,...,...,...,...,...,...,...
13189,1168,2632,2415,19,2139.0,0.726839,0.726839
13190,1252,501,13,390,408.0,0.127779,0.127779
13191,575,1252,2660,1601,1014.0,0.384920,0.384920
13192,251,390,331,528,297.0,0.007118,0.007118


In [8]:
indices = range(df.shape[0])

### Individual Classifiers are LSTMS

In [9]:
train, test, indices_train, indices_test = train_test_split(df, indices, test_size=0.2)

new_deception_train = train["Input.deception_quadrant"].copy()
new_deception_train['Input.deception_quadrant'] = train["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_train_deception = new_deception_train['Input.deception_quadrant'].tolist()
y_train_rapport = train['Answer.3rapport.yes_label'].tolist()
y_train_share_information = train['Answer.4shareinformation.yes_label'].tolist()
y_train_reasoning = train['Answer.2reasoning.yes_label'].tolist()
y_train_gamemove = train['Answer.1gamemove.yes_label'].tolist()

X_train_col = train['Input.full_text']

new_deception_test = test["Input.deception_quadrant"].copy()
new_deception_test['Input.deception_quadrant'] = test["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y_test_deception = new_deception_test['Input.deception_quadrant'].tolist()
y_test_rapport = test['Answer.3rapport.yes_label'].tolist()
y_test_share_information = test['Answer.4shareinformation.yes_label'].tolist()
y_test_reasoning = test['Answer.2reasoning.yes_label'].tolist()
y_test_gamemove = test['Answer.1gamemove.yes_label'].tolist()

X_test_col = test['Input.full_text']

In [10]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

In [11]:
le = LabelEncoder()

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_train_rapport = le.fit_transform(y_train_rapport)
y_train_rapport = y_train_rapport.reshape(-1,1)

y_train_share_information = le.fit_transform(y_train_share_information)
y_train_share_information = y_train_share_information.reshape(-1,1)

y_train_reasoning = le.fit_transform(y_train_reasoning)
y_train_reasoning = y_train_reasoning.reshape(-1,1)

y_train_gamemove = le.fit_transform(y_train_gamemove)
y_train_gamemove = y_train_gamemove.reshape(-1,1)

y_train_deception = le.fit_transform(y_train_deception)
y_train_deception = y_train_deception.reshape(-1,1)

y_test_rapport = le.fit_transform(y_test_rapport)
y_test_rapport = y_test_rapport.reshape(-1,1)

y_test_share_information = le.fit_transform(y_test_share_information)
y_test_share_information = y_test_share_information.reshape(-1,1)

y_test_reasoning = le.fit_transform(y_test_reasoning)
y_test_reasoning = y_test_reasoning.reshape(-1,1)

y_test_gamemove = le.fit_transform(y_test_gamemove)
y_test_gamemove = y_test_gamemove.reshape(-1,1)

y_test_deception = le.fit_transform(y_test_deception)
y_test_deception = y_test_deception.reshape(-1,1)

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [12]:
max_words = 1000
max_len = 220

tok = Tokenizer(num_words=max_words)

tok.fit_on_texts(X_train_col)
X_train_sequences = tok.texts_to_sequences(X_train_col)
X_train = pad_sequences(X_train_sequences, maxlen=max_len)

# tok.fit_on_texts(X_test_col)
X_test_sequences = tok.texts_to_sequences(X_test_col)
X_test = pad_sequences(X_test_sequences, maxlen=max_len)

In [13]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)
def create_lstm():
    Inp = Input(name='inputs', shape=[max_len])
    x = Embedding(max_words, 50, input_length=max_len)(Inp)
    x = LSTM(64, name='LSTM_01')(x)
    x = Dropout(0.5, name='Dropout')(x)
    x = Dense(128, activation='relu',name='Dense_01')(x)
    # x = Dropout(0.5,name='Dropout')(x)
    out = Dense(1,activation='sigmoid', name='output')(x)
    model = Model(inputs=Inp, outputs=out)
    
    return model

### Construct individual LSTM models

In [14]:
# Rapport model
rapport_model = create_lstm()
rapport_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
rapport_model.fit(X_train,y_train_rapport,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_rapport), callbacks=[early_stop])

Epoch 1/15
72/72 [==============================] - 2s 27ms/step - loss: 0.4287 - accuracy: 0.8557 - f1_m: 0.9226 - recall_m: 0.9954 - precision_m: 0.8609 - val_loss: 0.3904 - val_accuracy: 0.8769 - val_f1_m: 0.9340 - val_recall_m: 1.0000 - val_precision_m: 0.8767
Epoch 2/15
72/72 [==============================] - 1s 21ms/step - loss: 0.4127 - accuracy: 0.8590 - f1_m: 0.9249 - recall_m: 1.0000 - precision_m: 0.8609 - val_loss: 0.3853 - val_accuracy: 0.8769 - val_f1_m: 0.9340 - val_recall_m: 1.0000 - val_precision_m: 0.8767
Epoch 3/15
72/72 [==============================] - 1s 20ms/step - loss: 0.4094 - accuracy: 0.8590 - f1_m: 0.9249 - recall_m: 1.0000 - precision_m: 0.8609 - val_loss: 0.3882 - val_accuracy: 0.8769 - val_f1_m: 0.9340 - val_recall_m: 1.0000 - val_precision_m: 0.8767


In [15]:
rapport_pred = rapport_model.predict(X_train)
rapport_pred_test = rapport_model.predict(X_test)

In [16]:
rapport_pred_test_round = rapport_pred_test.round()
precision_recall_fscore_support(y_test_rapport, rapport_pred_test_round, average='macro')

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4384344766930519, 0.5, 0.46719775070290537, None)

In [17]:
# Game move model
gamemove_model = create_lstm()
gamemove_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
gamemove_model.fit(X_train,y_train_gamemove,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_gamemove), callbacks=[early_stop])

Epoch 1/15
72/72 [==============================] - 2s 25ms/step - loss: 0.2858 - accuracy: 0.9232 - f1_m: 0.9595 - recall_m: 0.9933 - precision_m: 0.9300 - val_loss: 0.2405 - val_accuracy: 0.9362 - val_f1_m: 0.9669 - val_recall_m: 1.0000 - val_precision_m: 0.9363
Epoch 2/15
72/72 [==============================] - 2s 22ms/step - loss: 0.2593 - accuracy: 0.9294 - f1_m: 0.9638 - recall_m: 1.0000 - precision_m: 0.9303 - val_loss: 0.2370 - val_accuracy: 0.9362 - val_f1_m: 0.9669 - val_recall_m: 1.0000 - val_precision_m: 0.9363
Epoch 3/15
72/72 [==============================] - 2s 24ms/step - loss: 0.2570 - accuracy: 0.9294 - f1_m: 0.9618 - recall_m: 1.0000 - precision_m: 0.9270 - val_loss: 0.2527 - val_accuracy: 0.9362 - val_f1_m: 0.9669 - val_recall_m: 1.0000 - val_precision_m: 0.9363


In [18]:
gamemove_pred = gamemove_model.predict(X_train)
gamemove_pred_test = gamemove_model.predict(X_test)

In [19]:
gamemove_pred_test_round = gamemove_pred_test.round()
precision_recall_fscore_support(y_test_gamemove, gamemove_pred_test_round, average='macro')

(0.46811785400175904, 0.5, 0.48353395412218947, None)

In [20]:
# Reasoning model
reasoning_model = create_lstm()
reasoning_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
reasoning_model.fit(X_train,y_train_reasoning,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_reasoning), callbacks=[early_stop])

Epoch 1/15
72/72 [==============================] - 2s 26ms/step - loss: 0.4774 - accuracy: 0.8259 - f1_m: 0.9052 - recall_m: 0.9964 - precision_m: 0.8308 - val_loss: 0.4544 - val_accuracy: 0.8347 - val_f1_m: 0.9097 - val_recall_m: 1.0000 - val_precision_m: 0.8347
Epoch 2/15
72/72 [==============================] - 2s 25ms/step - loss: 0.4647 - accuracy: 0.8279 - f1_m: 0.9048 - recall_m: 1.0000 - precision_m: 0.8268 - val_loss: 0.4482 - val_accuracy: 0.8347 - val_f1_m: 0.9097 - val_recall_m: 1.0000 - val_precision_m: 0.8347
Epoch 3/15
72/72 [==============================] - 1s 20ms/step - loss: 0.4589 - accuracy: 0.8279 - f1_m: 0.9066 - recall_m: 1.0000 - precision_m: 0.8302 - val_loss: 0.4639 - val_accuracy: 0.8347 - val_f1_m: 0.9097 - val_recall_m: 1.0000 - val_precision_m: 0.8347


In [21]:
reasoning_pred = reasoning_model.predict(X_train)
reasoning_pred_test = reasoning_model.predict(X_test)

In [22]:
reasoning_pred_test_round = reasoning_pred_test.round()
precision_recall_fscore_support(y_test_reasoning, reasoning_pred_test_round, average='macro')

(0.4173262972735268, 0.5, 0.4549376797698946, None)

In [23]:
# Share Information model
shareinfo_model = create_lstm()
shareinfo_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
shareinfo_model.fit(X_train,y_train_share_information,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_share_information), callbacks=[early_stop])

Epoch 1/15
72/72 [==============================] - 2s 26ms/step - loss: 0.4662 - accuracy: 0.8316 - f1_m: 0.9049 - recall_m: 0.9886 - precision_m: 0.8413 - val_loss: 0.4382 - val_accuracy: 0.8461 - val_f1_m: 0.9162 - val_recall_m: 1.0000 - val_precision_m: 0.8460
Epoch 2/15
72/72 [==============================] - 2s 21ms/step - loss: 0.4477 - accuracy: 0.8392 - f1_m: 0.9115 - recall_m: 1.0000 - precision_m: 0.8380 - val_loss: 0.4382 - val_accuracy: 0.8461 - val_f1_m: 0.9162 - val_recall_m: 1.0000 - val_precision_m: 0.8460


In [24]:
shareinfo_pred = shareinfo_model.predict(X_train)
shareinfo_pred_test = shareinfo_model.predict(X_test)

In [25]:
shareinfo_pred_test_round = shareinfo_pred_test.round()
precision_recall_fscore_support(y_test_share_information, shareinfo_pred_test_round, average='macro')

(0.4230430958663149, 0.5, 0.45831348261076704, None)

In [26]:
# Deception model
deception_model = create_lstm()
deception_model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy', f1_m, recall_m, precision_m])
deception_model.fit(X_train,y_train_deception,
          batch_size=128,
          epochs=15,
          validation_data=(X_test, y_test_deception), callbacks=[early_stop])

Epoch 1/15
72/72 [==============================] - 2s 26ms/step - loss: 0.2292 - accuracy: 0.9449 - f1_m: 0.9710 - recall_m: 0.9935 - precision_m: 0.9515 - val_loss: 0.1981 - val_accuracy: 0.9512 - val_f1_m: 0.9750 - val_recall_m: 1.0000 - val_precision_m: 0.9514
Epoch 2/15
72/72 [==============================] - 2s 21ms/step - loss: 0.2006 - accuracy: 0.9508 - f1_m: 0.9750 - recall_m: 1.0000 - precision_m: 0.9515 - val_loss: 0.1988 - val_accuracy: 0.9512 - val_f1_m: 0.9750 - val_recall_m: 1.0000 - val_precision_m: 0.9514


In [27]:
deception_pred = deception_model.predict(X_train)
deception_pred_test = deception_model.predict(X_test)
deception_pred_test_round = deception_pred_test.round()
precision_recall_fscore_support(y_test_deception, deception_pred_test_round, average='macro')

(0.47559366754617416, 0.5, 0.48749154834347536, None)

### One hot encodings

In [28]:
# Train encodings
pred_df_arr_full = []
pred_df_arr = []
for i in range(0, len(gamemove_pred)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = gamemove_pred[i][0]
    pred_obj_1['reasoning'] = reasoning_pred[i][0]
    pred_obj_1['shareinfo'] = shareinfo_pred[i][0]
    pred_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = rapport_pred[i][0]
    pred_df_arr_full.append(pred_obj_2)
    
pred_df_full = pd.DataFrame(pred_df_arr_full)
pred_df = pd.DataFrame(pred_df_arr)

In [29]:
# Test encodings
pred_test_df_arr_full = []
pred_test_df_arr = []

for i in range(0, len(gamemove_pred_test)):
    pred_obj_1 = {}
    pred_obj_1['gamemove'] = gamemove_pred_test[i][0]
    pred_obj_1['reasoning'] = reasoning_pred_test[i][0]
    pred_obj_1['shareinfo'] = shareinfo_pred_test[i][0]
    pred_test_df_arr.append(pred_obj_1)
    
    pred_obj_2 = pred_obj_1.copy()
    pred_obj_2['rapport'] = rapport_pred_test[i][0]
    pred_test_df_arr_full.append(pred_obj_2)
    
pred_test_df_full = pd.DataFrame(pred_test_df_arr_full)
pred_test_df = pd.DataFrame(pred_test_df_arr)

### Joint Model with one hot encoding 

In [30]:
def create_joint_model(df):
    inputB = Input(shape=(df.shape[1],))
    c = Dense(2, activation='relu')(inputB)
    c = Dense(4, activation='relu')(c)
    c = Dense(1, activation='softmax')(c)
    full_model = Model(inputs=inputB, outputs=c)

    full_model.compile(loss='binary_crossentropy', optimizer='adam', 
                          metrics=['acc',f1_m,precision_m, recall_m])
    
    return full_model

In [31]:
print('Joint full model with one hot encoding, predicting deception')
joint_full_model = create_joint_model(pred_df_full)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_test_df_full,y_test_deception), callbacks=[callback])

Joint full model with one hot encoding, predicting deception
Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 10        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.7497 - acc: 0.9508 - f1_m: 0.9738 - precision_m: 0.9495 - recall_m: 1.0000 - val_loss: 0.7444 - val_acc: 0.9512 - val_

In [32]:
joint_predict = joint_full_model.predict(pred_test_df_full)
joint_predict_round = []
for a in joint_predict:
    joint_predict_round.append([np.argmax(a)])
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.024406332453825858, 0.5, 0.04654088050314466, None)

In [33]:
print('Joint full model with one hot encoding, predicting rapport')
joint_full_model = create_joint_model(pred_df)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df, y=y_train_rapport, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_test_df,y_test_rapport), callbacks=[callback])

Joint full model with one hot encoding, predicting rapport
Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 2.1502 - acc: 0.8590 - f1_m: 0.9241 - precision_m: 0.8599 - recall_m: 1.0000 - val_loss: 1.8777 - val_acc: 0.8769 - val_f1

In [34]:
joint_predict = joint_full_model.predict(pred_test_df)
joint_predict_round = []
for a in joint_predict:
    joint_predict_round.append([np.argmax(a)])
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.06156552330694811, 0.5, 0.1096319498825372, None)

### Joint Model by Keras Concatenate

In [35]:
def create_concatenate_keras_model(models_arr):
    commonInput = Input(shape=[max_len])

    input_model_arr = []
    for model in models_arr: 
        outmodel = model(commonInput)
        input_model_arr.append(outmodel)
    
    mergedOut = Concatenate()(input_model_arr)

    mergedOut = Flatten()(mergedOut)    
    mergedOut = Dense(256, activation='relu')(mergedOut)
    mergedOut = Dropout(.5)(mergedOut)
    mergedOut = Dense(128, activation='relu')(mergedOut)
    mergedOut = Dropout(.35)(mergedOut)
    mergedOut = Dense(1, activation='softmax')(mergedOut)  #Cuz binary

    mergedModel = Model(commonInput, mergedOut)
    mergedModel.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy', f1_m, recall_m, precision_m])
    
    return mergedModel

In [36]:
print('Joint model by concatenate, predicting deception')
merged_model = create_concatenate_keras_model([gamemove_model, reasoning_model, shareinfo_model, rapport_model])
merged_model.summary()
merged_model.fit(X_train,y_train_deception,
          batch_size=128,
          epochs=15,
          validation_data=(X_test,y_test_deception), callbacks=[early_stop])

Joint model by concatenate, predicting deception
Model: "functional_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 220)]        0                                            
__________________________________________________________________________________________________
functional_3 (Functional)       (None, 1)            87889       input_3[0][0]                    
__________________________________________________________________________________________________
functional_5 (Functional)       (None, 1)            87889       input_3[0][0]                    
__________________________________________________________________________________________________
functional_7 (Functional)       (None, 1)            87889       input_3[0][0]                    
_____________________________________

In [37]:
print('Joint model by concatenate, predicting rapport')
merged_model = create_concatenate_keras_model([gamemove_model, reasoning_model, shareinfo_model])
merged_model.summary()
merged_model.fit(X_train,y_train_rapport,
          batch_size=128,
          epochs=15,
          validation_data=(X_test,y_test_rapport), callbacks=[early_stop])

Joint model by concatenate, predicting rapport
Model: "functional_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 220)]        0                                            
__________________________________________________________________________________________________
functional_3 (Functional)       (None, 1)            87889       input_4[0][0]                    
__________________________________________________________________________________________________
functional_5 (Functional)       (None, 1)            87889       input_4[0][0]                    
__________________________________________________________________________________________________
functional_7 (Functional)       (None, 1)            87889       input_4[0][0]                    
_______________________________________

## Weighted against Throughput & WorkTime

In [38]:
# Train weighted encodings
pred_df_full_throughput = pred_df_full.copy()
pred_df_full_worktime = pred_df_full.copy()
pred_df_throughput = pred_df.copy()
pred_df_worktime = pred_df.copy()

throughput_values = df_throughput['var_throughput'].take(indices_train).values
pred_df_full_throughput = pred_df_full_throughput.mul(throughput_values, axis=0)
pred_df_throughput = pred_df_throughput.mul(throughput_values, axis=0)

worktime_values = df_worktime['var_worktime'].take(indices_train).values
pred_df_full_worktime = pred_df_full_worktime.mul(worktime_values, axis=0)
pred_df_worktime = pred_df_worktime.mul(worktime_values, axis=0)

In [39]:
pred_df_full.head()

,gamemove,reasoning,shareinfo,rapport
0,0.904174,0.916218,0.811955,0.933604
1,0.897872,0.918377,0.803144,0.944711
2,0.895877,0.904362,0.790460,0.939799
3,0.901822,0.880043,0.797400,0.923424
4,0.884904,0.909780,0.809640,0.924574


In [40]:
pred_df_full_throughput.head()

,gamemove,reasoning,shareinfo,rapport
0,0.004403,0.004461,0.003954,0.004546
1,0.754537,0.771769,0.674931,0.793899
2,0.417646,0.421602,0.368502,0.438122
3,0.791001,0.771898,0.699411,0.809948
4,0.432799,0.444966,0.395988,0.452201


In [41]:
# Test weighted encodings
pred_df_full_throughput_test = pred_test_df_full.copy()
pred_df_full_worktime_test = pred_test_df_full.copy()
pred_df_throughput_test = pred_test_df.copy()
pred_df_worktime_test = pred_test_df.copy()

throughput_values_test = df_throughput['var_throughput'].take(indices_test).values
pred_df_full_throughput_test = pred_df_full_throughput_test.mul(throughput_values_test, axis=0)
pred_df_throughput_test = pred_df_throughput_test.mul(throughput_values_test, axis=0)

worktime_values_test = df_worktime['var_worktime'].take(indices_test).values
pred_df_full_worktime_test = pred_df_full_worktime_test.mul(worktime_values_test, axis=0)
pred_df_worktime_test = pred_df_worktime_test.mul(worktime_values_test, axis=0)

In [42]:
pred_df_throughput_test.head()

,gamemove,reasoning,shareinfo
0,0.111068,0.107816,0.101914
1,0.842065,0.825836,0.746711
2,0.526439,0.529091,0.463505
3,0.011915,0.011823,0.010660
4,0.605660,0.624752,0.543460


In [43]:
pred_test_df.head()

,gamemove,reasoning,shareinfo
0,0.857640,0.832532,0.786955
1,0.890549,0.873386,0.789704
2,0.890432,0.894917,0.783984
3,0.893293,0.886343,0.799185
4,0.885502,0.913415,0.794562


## Throughput only

### Joint Model with one hot encoding 

In [44]:
def create_joint_model(df):
    inputB = Input(shape=(df.shape[1],))
    c = Dense(2, activation='relu')(inputB)
    c = Dense(4, activation='relu')(c)
    c = Dense(1, activation='softmax')(c)
    full_model = Model(inputs=inputB, outputs=c)

    full_model.compile(loss='binary_crossentropy', optimizer='adam', 
                          metrics=['acc',f1_m,precision_m, recall_m])
    
    return full_model

In [45]:
print('Joint full model with one hot encoding, predicting deception, weighted by throughput')
joint_full_model = create_joint_model(pred_df_full_throughput)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_throughput, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_df_full_throughput_test,y_test_deception), callbacks=[callback])

Joint full model with one hot encoding, predicting deception, weighted by throughput
Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_13 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.7497 - acc: 0.9508 - f1_m: 0.9748 - precision_m: 0.9512 - recall_m: 1.0000 - val_loss: 0.7444 

In [46]:
joint_predict = joint_full_model.predict(pred_df_full_throughput_test)
joint_predict_round = []
for a in joint_predict:
    joint_predict_round.append([np.argmax(a)])
precision_recall_fscore_support(y_test_deception, np.array(joint_predict_round), average='macro')

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.024406332453825858, 0.5, 0.04654088050314466, None)

In [47]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)

In [48]:
print('Joint full model with one hot encoding, predicting rapport, weighted by throughput')
joint_full_model = create_joint_model(pred_df_throughput)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_throughput, y=y_train_rapport, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_df_throughput_test,y_test_rapport), callbacks=[callback])

Joint full model with one hot encoding, predicting rapport, weighted by throughput
Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 2.1502 - acc: 0.8590 - f1_m: 0.9240 - precision_m: 0.8599 - recall_m: 1.0000 - val_loss: 1.8777 - 

In [49]:
joint_predict = joint_full_model.predict(pred_df_throughput_test)
joint_predict_round = []
for a in joint_predict:
    joint_predict_round.append([np.argmax(a)])
precision_recall_fscore_support(y_test_rapport, np.array(joint_predict_round), average='macro')

/home/kaiyuan/anaconda3/envs/TF2/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.06156552330694811, 0.5, 0.1096319498825372, None)

## WorkTime only

### Joint Model with one hot encoding 

In [50]:
def create_joint_model(df):
    inputB = Input(shape=(df.shape[1],))
    c = Dense(2, activation='relu')(inputB)
    c = Dense(4, activation='relu')(c)
    c = Dense(1, activation='softmax')(c)
    full_model = Model(inputs=inputB, outputs=c)

    full_model.compile(loss='binary_crossentropy', optimizer='adam', 
                          metrics=['acc',f1_m,precision_m, recall_m])
    
    return full_model

In [51]:
print('Joint full model with one hot encoding, predicting deception, weighted by throughput')
joint_full_model = create_joint_model(pred_df_full_worktime)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_full_worktime, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_df_full_worktime_test,y_test_deception), callbacks=[callback])

Joint full model with one hot encoding, predicting deception, weighted by throughput
Model: "functional_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_19 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 5         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 0.7497 - acc: 0.9508 - f1_m: 0.9748 - precision_m: 0.9512 - recall_m: 1.0000 - val_loss: 0.7444 

In [52]:
y_test_rapport = np.asarray(y_test_rapport)
y_train_rapport = np.asarray(y_train_rapport)


In [53]:
print('Joint full model with one hot encoding, predicting rapport, weighted by throughput')
joint_full_model = create_joint_model(pred_df_worktime)
joint_full_model.summary()
history = joint_full_model.fit(x=pred_df_worktime, y=y_train_rapport, epochs=32, 
                    batch_size=64, 
                    validation_data=(pred_df_worktime_test,y_test_rapport), callbacks=[callback])

Joint full model with one hot encoding, predicting rapport, weighted by throughput
Model: "functional_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_22 (Dense)             (None, 4)                 12        
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
143/143 [==============================] - 0s 3ms/step - loss: 2.1502 - acc: 0.8590 - f1_m: 0.9240 - precision_m: 0.8599 - recall_m: 1.0000 - val_loss: 1.8777 - 